In [1]:
import numpy as np
import pandas as pd

In [2]:
import prepostprocessing.cleaning as cleaning
import prepostprocessing.pre_processing as preproc
from qapf import qapf
from qapf import cipw

___

# CIPWFULL normalized data

## Convert to CIPWFULL format

**I used an arbitrary version of the mineralogical data, not sure if this is the correct one** -->is a good one

In [ ]:
mineralogy_original = pd.read_excel("../_CIPW/CIPW/AREA5/interpolated_data.xlsx", index_col=0)
# Only iterested in sample number and oxides
mineralogy = mineralogy_original[['SiO2', 'TiO2', 'Al2O3', 'Fe2O3','FeO', 'MnO', 'MgO', 'CaO', 'Na2O', 'K2O', 'P2O5', 'l.i.', 'oth']]
mineralogy.columns

In [ ]:
mineralogy = mineralogy.dropna()

In [ ]:
# Check for closed sum
assert all(np.isclose(mineralogy.sum(axis=1), 100.0))

In [ ]:
# Remove 'l.i.'' and 'oth' columns and immediately normalize data
mineralogy = preproc.normalize(mineralogy.drop(["l.i.", "oth"], axis=1))

In [ ]:
# Check for closed sum
assert all(np.isclose(mineralogy.sum(axis=1), 100.0))

In [ ]:
mineralogy.head()

In [ ]:
mineralogy1 = mineralogy.iloc[0:45000]
mineralogy2 = mineralogy.iloc[45000:90000]
mineralogy3 = mineralogy.iloc[90000:135000]
mineralogy4 = mineralogy.iloc[135000:180000]



In [11]:
cipw.convert_to_CIPWFULL_format(mineralogy1, "../_CIPW/CIPW/AREA5/area5_1.txt", dataset_name="Vistelius (1995)")
cipw.convert_to_CIPWFULL_format(mineralogy2, "../_CIPW/CIPW/AREA5/area5_2.txt", dataset_name="Vistelius (1995)")
cipw.convert_to_CIPWFULL_format(mineralogy3, "../_CIPW/CIPW/AREA5/area5_3.txt", dataset_name="Vistelius (1995)")
cipw.convert_to_CIPWFULL_format(mineralogy4, "../_CIPW/CIPW/AREA5/area5_4.txt", dataset_name="Vistelius (1995)")

_____

## Extract results from CIPWFULL results file

**I only selected quartz, plagioclase and alkali feldspar here as they are of interest to us for the QAPF classification**  
**Feel free to extract addiotnal minerals if they would be of any interest to you**

**Not sure what to do with albite in the final assignments as by the book it should be accounted for as an alkali feldspar and thus be included in the sum of plagioclase but then the resulting plagioclase contents seem quite low.   
This is something you might look into although the method used here seems to be correct according to te resulting QAPF diagram.**

--> indeed when reading the paper of Streckeisen (and other sources), albite should be accounted for as an alkali feldspar, also saw that the datapoints were generally located more to the right side (plagioclase), maybe an overestimation of plagioclase?

--> I changed the settings below so that albite belongs to the alkali feldspars, Think this figure should be more correct according to the papers, but the cotent of plagioclase is indeed quite low. 

In [25]:
df_final = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA5/area5_1_results_excel.txt")

In [26]:
df_final.to_excel("../_CIPW/CIPW/AREA5/qapf.xlsx", index=True)

In [24]:
df_final

,Q,P,K
SAMPLE,,,
5249,27.208198,36.628164,27.326432
5250,27.167389,36.656974,27.315743
5251,27.126472,36.685809,27.305004
5252,27.085567,36.714535,27.294222
5253,27.044643,36.743102,27.283505
...,...,...,...
111751,30.801493,32.987579,27.919390
111752,30.846561,32.971570,27.937194
111753,30.890490,32.956188,27.954603


In [14]:
df_final = pd.concat([df_final1, df_final2, df_final3, df_final4])

In [15]:
df_final

,Q,P,K
SAMPLE,,,
5249,27.208198,36.628164,27.326432
5250,27.167389,36.656974,27.315743
5251,27.126472,36.685809,27.305004
5252,27.085567,36.714535,27.294222
5253,27.044643,36.743102,27.283505
...,...,...,...
363036,29.914730,34.397391,28.094479
363037,29.915543,34.396695,28.094040
363038,29.916504,34.395953,28.093617


## QAPF classification

In [27]:
df_final["QAPF"] = qapf.check_QAPF_region(df_final)

In [28]:
df_final["QAPF"].unique()

array(['monzo granite'], dtype=object)

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

fig = px.scatter_ternary(df_final, a="Q", b="K", c="P", color="QAPF")

fig.write_image(f"../_FIGURES/geomap/qapf_ternary/area5_QAPF.pdf")
fig.show()

In [18]:
df_final.to_excel("../_CIPW/CIPW/AREA5/qapf.xlsx", index=True)

-------